[View in Colaboratory](https://colab.research.google.com/github/hschoi1/rich_latent/blob/master/MNIST_visualizaitons.ipynb)

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
import itertools
import pdb
from sklearn.utils import shuffle
from sklearn.metrics import average_precision_score, roc_auc_score
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors
tf.logging.set_verbosity(tf.logging.INFO)
tf.set_random_seed(0)

In [0]:

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/MNIST_data/", one_hot=False)

train_data = mnist.train.images.astype(float)
train_labels = mnist.train.labels.astype(float)
from sklearn.utils import shuffle
train_data, train_labels = shuffle(train_data, train_labels, random_state=0)
numbers = []
for i in range(10):
    numbers.append(train_data[train_labels==i, :])

test_data = train_data[-100:]
test_labels = train_labels[-100:]

train_data = train_data[:-100]
train_labels = train_labels[:-100]

In [0]:
def ensemble_mean_var(ensemble_results):
    M = len(ensemble_results)
    final_mean = 0
    final_var = 0
    for i in range(M):
        mean, var = ensemble_results[i]
        final_mean += mean / M
        final_var += (var + np.square(mean))/ M
    final_var -= np.square(final_mean)
    return final_mean, final_var


# log p(x|zk) specified by the paper
def loss_fct(mean, var, labels):
    loss = 0
    loss += tf.log(var) / 2
    loss += tf.square(labels - mean) / (2 * var)
    return loss


def get_batch(data_x, batch_size=100):
    batch_n = len(data_x) // batch_size
    for i in range(batch_n):
        batch_x = data_x[i * batch_size:(i + 1) * batch_size]
        yield batch_x
        
def mean_var(results):
    results = tf.cast(results, dtype=tf.float32)
    predictive_mean, predictive_variance = tf.nn.moments(results, axes=[0])
    return predictive_mean, predictive_variance


In [0]:



num_models = 5
ensemble_results = []
adv_ensemble_results = []
adv_ensemble_results2 = []
adv_latent_prob_results1 = []
adv_latent_prob_results2 = []
adv_elbo = []
adv_elbo2 = []

def make_encoder(x, z_size=2):  
    x = tf.cast(x, dtype=tf.float32)
    net = make_enc_nn(x, z_size * 2)
    #return tfd.MultivariateNormalDiag(loc=net[..., :z_size], scale_diag=tf.nn.softplus(net[..., z_size:]))
    return net[..., :z_size], tf.nn.softplus(net[..., z_size:])


def make_decoder(z, x_shape=784):
    net = make_dec_nn(z, x_shape)
    #mu_net = net[...,:x_shape]
    #sigma_net = net[...,x_shape:]
    return tfd.Independent(tfd.Bernoulli(logits=net))

def make_enc_nn(x, out_size, hidden_size=(128,128)):
    net = x
    for h in hidden_size:
        net = tf.layers.dense(inputs=net, units=h, activation=tf.nn.relu)
    return tf.layers.dense(inputs=net, units=out_size)


def make_dec_nn(x, out_size, hidden_size=(256,256,256)):
    net = x
    for h in hidden_size:
        net = tf.layers.dense(inputs=net, units=h, activation=tf.nn.relu)
    return tf.layers.dense(inputs=net, units=out_size)



def make_arflow(z_dist, n_flows=8, hidden_size=(512,) * 2, invert=False):
    chain = list(itertools.chain.from_iterable([tfb.MaskedAutoregressiveFlow(
        shift_and_log_scale_fn=tfb.masked_autoregressive_default_template(hidden_size)),
                                                   tfb.Permute(np.random.permutation(2)), ] for _ in range(n_flows)))
    return tfd.TransformedDistribution(distribution=z_dist, bijector=tfb.Chain(chain[:-1]))

def make_prior(z_size=2):
    return make_arflow(tfd.MultivariateNormalDiag(loc=tf.zeros([z_size], dtype=tf.float32), scale_diag = tf.ones([z_size], dtype=tf.float32)))


# replicate paper

# check if it really works


tf.reset_default_graph()
with tf.Graph().as_default() as g:
    input_layer = tf.placeholder(tf.float32, shape=[None, 784], name='x')
    output_layer = tf.placeholder(tf.float32, shape=[None, 784], name='x')
    
    
    latent = make_prior()

    # sample z0
    with tf.variable_scope("encoder"):
      enc_mean, enc_std = make_encoder(input_layer)
    enc = tfd.MultivariateNormalDiag(loc=enc_mean, scale_diag=enc_std)
    enc_samples = enc.sample()
    
    
    posterior_samples = enc.sample(100)
    samples = [latent.bijector.inverse(posterior_samples)]
    samples.append(posterior_samples)
    
    '''
    #my_samples = enc.sample(100)
    latent_samples = latent.sample(100)
    samples = [latent_samples]
    
    # construct IAF flows
    
    z_tmp = latent.bijector.inverse(latent_samples)
    samples = [z_tmp]

    names = [enc.name]
    names = [latent.name]
    # transform back to the prior
#     for bijector in latent.bijector.bijectors:
#         latent_samples = bijector.inverse(latent_samples)
#         samples.append(latent_samples) 
        
#     samples = samples[::-1]
    for bijector in reversed(latent.bijector.bijectors):
         z_tmp = bijector.forward(z_tmp)
         samples.append(z_tmp)
         names.append(bijector.name)
    # import pdb; pdb.set_trace()
    '''
    # reconstruct input

    with tf.variable_scope("decoder"):
      recons = make_decoder(enc_samples)

    recons_mean = recons.mean()
    recons_var = recons.variance()
   
  
    # elbo2  is reconstruction loss
    # elbo1+elbo3 is "kl divergence"
    
    
    elbo1_list = -latent.log_prob(enc_samples)
    elbo1 = tf.reduce_mean(elbo1_list)
    elbo2_list = -recons.log_prob(output_layer)
    elbo2 = tf.reduce_mean(elbo2_list)
    elbo3_list = enc.log_prob(enc_samples)
    elbo3 = tf.reduce_mean(elbo3_list)
    elbo_list = elbo1_list+elbo2_list+elbo3_list
    elbo = tf.reduce_mean(elbo_list)
    
    gradients = tf.gradients(elbo, input_layer)
    adv_examples = input_layer + 0.3*tf.sign(gradients)[0]
    
    
    random_noise = tfd.Uniform(low=0.0, high=1.0)
    random_noise_samples = random_noise.sample(sample_shape=(100,784))
    adv_examples2 = random_noise_samples - 0.3*tf.sign(gradients)[0]
    
    gaussian_random_noise = tfd.Normal(loc=0.0, scale=1.0)
    gaussian_random_noise_samples = gaussian_random_noise.sample(sample_shape=(100,784))
    
    
    
    
    #adv_examples = input_layer 
    
    with tf.variable_scope("encoder", reuse=True):
      adv_enc_mean, adv_enc_std = make_encoder(adv_examples)
    with tf.variable_scope("encoder", reuse=True):  
      adv_enc_mean2, adv_enc_std2 = make_encoder(adv_examples2)
    with tf.variable_scope("encoder", reuse=True):
      adv_enc_mean3, adv_enc_std3 = make_encoder(gaussian_random_noise_samples)
      
    adv_enc = tfd.MultivariateNormalDiag(loc=adv_enc_mean, scale_diag=adv_enc_std)
    adv_enc2 = tfd.MultivariateNormalDiag(loc=adv_enc_mean2, scale_diag=adv_enc_std2)
    adv_enc3 = tfd.MultivariateNormalDiag(loc=adv_enc_mean3, scale_diag=adv_enc_std3)
    
    adv_enc_samples = adv_enc.sample()
    adv_enc_samples2 = adv_enc2.sample()
    adv_enc_samples3 = adv_enc3.sample()
    
    adv_posterior_samples = adv_enc.sample(100)
    adv_posterior_samples2 = adv_enc2.sample(100)
    adv_posterior_samples3 = adv_enc3.sample(100)
    
    adv_samples = [latent.bijector.inverse(adv_posterior_samples)]
    adv_samples2 = [latent.bijector.inverse(adv_posterior_samples2)]
    adv_samples3 = [latent.bijector.inverse(adv_posterior_samples3)]

    adv_samples.append(adv_posterior_samples)
    adv_samples2.append(adv_posterior_samples2)
    adv_samples3.append(adv_posterior_samples3)
    
    with tf.variable_scope("decoder", reuse=True):
      adv_recons = make_decoder(adv_enc_samples)
    with tf.variable_scope("decoder", reuse=True):
      adv_recons2 = make_decoder(adv_enc_samples2)
    with tf.variable_scope("decoder", reuse=True):
      adv_recons3 = make_decoder(adv_enc_samples3)  

    adv_recons_mean = adv_recons.mean()
    adv_recons_mean2 = adv_recons2.mean()
    adv_recons_mean3 = adv_recons3.mean()
    
    adv_recons_var = adv_recons.variance()
    adv_recons_var2 = adv_recons2.variance()
    adv_recons_var3 = adv_recons3.variance()
    
    adv_elbo1_list = - latent.log_prob(adv_enc_samples)
    adv_elbo1 = tf.reduce_mean(adv_elbo1_list)
    adv_elbo2_list = - adv_recons.log_prob(output_layer)
    adv_elbo2 = tf.reduce_mean(adv_elbo2_list)
    adv_elbo3_list = adv_enc.log_prob(adv_enc_samples)
    adv_elbo3 = tf.reduce_mean(adv_elbo3_list)
    adv_elbo_list = adv_elbo1_list + adv_elbo2_list + adv_elbo3_list
    adv_elbo = adv_elbo1 + adv_elbo2 + adv_elbo3
    
    adv_elbo1_list2 = - latent.log_prob(adv_enc_samples2)
    adv_elbo12 = tf.reduce_mean(adv_elbo1_list2)
    adv_elbo2_list2 = - adv_recons2.log_prob(output_layer)
    adv_elbo22 = tf.reduce_mean(adv_elbo2_list2)
    adv_elbo3_list2 = adv_enc2.log_prob(adv_enc_samples2)
    adv_elbo32 = tf.reduce_mean(adv_elbo3_list2)
    adv_elbo_list2 = adv_elbo1_list2 + adv_elbo2_list2 + adv_elbo3_list2
    adv_elbo2 = adv_elbo12 + adv_elbo22 + adv_elbo32
    
    optimizer = tf.train.AdamOptimizer(0.00001)
    train_op = optimizer.minimize(elbo)
    #f_train_op = optimizer.minimize(elbo, var_list=flows_variables+decoder_variables)


    tf.summary.scalar('loss', elbo)
    summ_op = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('./logs/train')
    saver = tf.train.Saver()
    init = tf.global_variables_initializer()
    g.finalize()

# train and visualize
with tf.Session(graph=g) as sess:
   
    train_writer.add_graph(g)
    
    for model in range(num_models):
        step = 0
        losses = []
        losses_1 = []
        losses_2 = []
        losses_3 = []
        global_step = []
        sess.run(init)
        
        for epoch in range(20):
        
            train_data = shuffle(train_data, random_state=0)
            print("model",str(model),"epoch",str(epoch))

            for batch_x in get_batch(train_data, 100):
                # train
                feed_dict = {input_layer: batch_x, output_layer:batch_x}
                #if step <= 5000:
                fetch_dict = {'loss': elbo, 'elbo1': elbo1, 'elbo2': elbo2, 'elbo3': elbo3, 'train': train_op, 'summary': summ_op}
                #else:
                #    fetch_dict = {'loss': elbo, 'elbo1': elbo1, 'elbo2': elbo2, 'elbo3': elbo3, 'train': f_train_op, 'summary': summ_op}
                result = sess.run(fetch_dict, feed_dict)
                # collect loss
                if step % 400 == 0:
                    train_writer.add_summary(result['summary'], step)
                    global_step.append(step)
                    losses.append(result['loss'])
                    losses_1.append(result['elbo1'])
                    losses_2.append(result['elbo2'])
                    losses_3.append(result['elbo3'])
                    print("loss at", step, ":", result['loss'], "        elbo1:", result["elbo1"], "  elbo2:", result["elbo2"], "  elbo3:", result["elbo3"])

                step += 1      

        # testing
          
        #f, arr = plt.subplots(1, 3, figsize=(5 * 2, 5))
        numbers_results=[]
        adv_numbers_results1=[]
        adv_numbers_results2=[]
        adv_numbers_results3=[]
        feed_dict = {input_layer:test_data, output_layer:test_data}
        #if step <= 3000:
        fetch_dict = {'recons_mean':recons_mean, 'recons_var':recons_var,'samples':samples, "enc_mean":enc_mean, "enc_std":enc_std,
                      'adv_recons_mean':adv_recons_mean, 'adv_recons_var':adv_recons_var,
                      'elbo_list':elbo_list,'elbo1_list':elbo1_list,'elbo2_list':elbo2_list,'elbo3_list':elbo3_list,
                     'adv_elbo_list':adv_elbo_list,'adv_elbo1_list':adv_elbo1_list,'adv_elbo2_list':adv_elbo2_list,'adv_elbo3_list':adv_elbo3_list,
                     'adv_examples':adv_examples, 'adv_examples2':adv_examples2,'adv_examples3':gaussian_random_noise_samples, 'adv_recons_mean2':adv_recons_mean2, 
                      'adv_recons_var2':adv_recons_var2,'adv_recons_mean3':adv_recons_mean3,'adv_recons_var3':adv_recons_var3, 'adv_elbo1_list2':adv_elbo1_list2,
                          'adv_elbo2_list2':adv_elbo2_list2,'adv_elbo3_list2':adv_elbo3_list2, 'adv_elbo_list2':adv_elbo_list2}

        test_results = sess.run(fetch_dict, feed_dict)
 

        

        num_layer = 2

        # run digits from training distribution for visualization of latent space
        for num_array in numbers:
            feed_dict = {input_layer: num_array[:100], output_layer:num_array[:100]}
            fetch_dict = {'samples': samples, 'adv_samples':adv_samples, 'adv_samples2':adv_samples2,'adv_samples3':adv_samples3,
                         'elbo_list':elbo_list, 'adv_elbo1_list':adv_elbo1_list, 'adv_elbo1_list2':adv_elbo1_list2}

            results = sess.run(fetch_dict, feed_dict)

            num_results = results["samples"]
            adv_num_results1 = results["adv_samples"]
            adv_latent_prob1 = results["adv_elbo1_list"]
            adv_num_results2 = results["adv_samples2"]
            adv_latent_prob2 = results["adv_elbo1_list2"]
            adv_num_results3 = results["adv_samples3"]
            

            numbers_results.append(num_results)
            adv_numbers_results1.append(adv_num_results1)
            adv_numbers_results2.append(adv_num_results2)
            adv_numbers_results3.append(adv_num_results3)
            adv_latent_prob_results1.append(adv_latent_prob1)
            adv_latent_prob_results2.append(adv_latent_prob2)


        # visualize latent of normal examples with color     
        f, arr = plt.subplots(1, num_layer, figsize=(5 * num_layer, 5))
        colors = cm.rainbow(np.linspace(0, 1, 10))

        for i in range(num_layer):
            for num, c in zip(range(10), colors):
                X1 = numbers_results[num][i]
                arr[i].scatter(X1[:, :100, 0], X1[:, :100, 1], s=1, color=c, label=num, alpha=0.5)
                arr[i].set_xlim(-15,15)
                arr[i].set_ylim(-15,15)
        plt.legend()
        plt.show()
        
        # visualize latent of adversarial examples1 with color
        f, arr = plt.subplots(1, num_layer, figsize=(5 * num_layer, 5))
        colors = cm.rainbow(np.linspace(0, 1, 10))

        for i in range(num_layer):
            for num, c in zip(range(10), colors):
                X1 = adv_numbers_results1[num][i]
                arr[i].scatter(X1[:, :100, 0], X1[:, :100, 1], s=1, color=c, label=num, alpha=0.5)
                arr[i].set_xlim(-40,20)
                arr[i].set_ylim(-10,50)
        plt.legend()
        plt.show()

        # visualize latent of adversarial examples2 with color
        f, arr = plt.subplots(1, num_layer, figsize=(5 * num_layer, 5))
        colors = cm.rainbow(np.linspace(0, 1, 10))

        for i in range(num_layer):
            for num, c in zip(range(10), colors):
                X1 = adv_numbers_results2[num][i]
                arr[i].scatter(X1[:, :100, 0], X1[:, :100, 1], s=1, color=c, label=num, alpha=0.5)
                arr[i].set_xlim(-10,10)
                arr[i].set_ylim(-10,10)
        plt.legend()
        plt.show()
   
  
        
        f, arr = plt.subplots(1, num_layer, figsize=(5 * num_layer, 5))
        colors = cm.rainbow(np.linspace(0, 1, 10))

        for i in range(num_layer):
            for num, c in zip(range(10), colors):
                X1 = adv_numbers_results3[num][i]
                arr[i].scatter(X1[:, :100, 0], X1[:, :100, 1], s=1, color=c, label=num, alpha=0.5)
                arr[i].set_xlim(-10,10)
                arr[i].set_ylim(-10,10)
        plt.legend()
        plt.show()



        f, arr = plt.subplots(1,4, figsize=(5 * num_layer, 5))
        arr[0].plot(losses[1:])
        arr[1].plot(losses_1[1:])
        arr[2].plot(losses_2[1:])
        arr[3].plot(losses_3[1:])
        plt.show()
          
        ensemble_results.append((test_results["recons_mean"],test_results["recons_var"]))
        adv_ensemble_results.append((test_results["adv_recons_mean"], test_results["adv_recons_var"]))
        adv_ensemble_results2.append((test_results["adv_recons_mean2"], test_results["adv_recons_var2"]))
        
        saver.save(sess, './logs/ckpt/model-%d.ckpt' % (epoch))

        
        
    # last single model input reconstruction
    f, arr = plt.subplots(4, 3, figsize=(15, 15), sharex='all', sharey='all')


    #normal points

    single_mean = test_results['recons_mean']
    single_var = test_results['recons_var']
    arr[0, 0].imshow(np.reshape(test_data[1],[28,28]))
    arr[0, 1].imshow(np.reshape(single_mean[1],[28,28]))
    arr[0, 2].imshow(np.ones((28,28))-np.reshape(single_var[1],[28,28]))

    #adversarial
    adv_single_mean = test_results['adv_recons_mean']
    adv_single_var = test_results['adv_recons_var']
    arr[1, 0].imshow(np.reshape(test_results['adv_examples'][1],[28,28]))
    arr[1, 1].imshow(np.reshape(adv_single_mean[1],[28,28]))
    arr[1, 2].imshow(np.ones((28,28))-np.reshape(adv_single_var[1],[28,28]))
    
    #adversarial2
    adv_single_mean2 = test_results['adv_recons_mean2']
    adv_single_var2 = test_results['adv_recons_var2']
    arr[2, 0].imshow(np.reshape(test_results['adv_examples2'][1],[28,28]))
    arr[2, 1].imshow(np.reshape(adv_single_mean2[1],[28,28]))
    arr[2, 2].imshow(np.ones((28,28))-np.reshape(adv_single_var2[1],[28,28]))
    
    #adversarial3
    adv_single_mean3 = test_results['adv_recons_mean3']
    adv_single_var3 = test_results['adv_recons_var3']
    arr[3, 0].imshow(np.reshape(test_results['adv_examples3'][1],[28,28]))
    arr[3, 1].imshow(np.reshape(adv_single_mean3[1],[28,28]))
    arr[3, 2].imshow(np.ones((28,28))-np.reshape(adv_single_var3[1],[28,28]))
    
    plt.show()

    # ensemble model input reconstruction


    f, arr = plt.subplots(3, 3, figsize=(15, 15), sharex='all', sharey='all')
    ensemble_mean, ensemble_var = ensemble_mean_var(ensemble_results)
    adv_ensemble_mean, adv_ensemble_var = ensemble_mean_var(adv_ensemble_results)
    adv_ensemble_mean2, adv_ensemble_var2 = ensemble_mean_var(adv_ensemble_results2)

    #normal points

    arr[0, 0].imshow(np.reshape(test_data[1],[28,28]))
    arr[0, 1].imshow(np.reshape(ensemble_mean[1],[28,28]))
    arr[0, 2].imshow(np.ones((28,28))-np.reshape(ensemble_var[1],[28,28]))

    #adversarial   

    arr[1, 0].imshow(np.reshape(test_results['adv_examples'][1],[28,28]))
    arr[1, 1].imshow(np.reshape(adv_ensemble_mean[1],[28,28]))
    arr[1, 2].imshow(np.ones((28,28))-np.reshape(adv_ensemble_var[1],[28,28]))
    
    
    #adversarial2
    
    arr[2, 0].imshow(np.reshape(test_results['adv_examples2'][1],[28,28]))
    arr[2, 1].imshow(np.reshape(adv_ensemble_mean2[1],[28,28]))
    arr[2, 2].imshow(np.ones((28,28))-np.reshape(adv_ensemble_var2[1],[28,28]))
    

    plt.show()
    
    
    f, arr = plt.subplots(4, 2, figsize=(15, 15))
    
    arr[0,0].hist(single_var)
    arr[0,1].hist(ensemble_var)
    
    arr[1,0].hist(adv_single_var)
    arr[1,1].hist(adv_ensemble_var)
    
    arr[2,0].hist(adv_single_var2)
    arr[2,1].hist(adv_ensemble_var2)
    
    arr[3,0].hist(test_results['elbo_list'])
    arr[3,1].hist(test_results['adv_elbo_list2'])
    plt.show()

